### 텐서플로 2.0 기본 모델링

In [3]:
import tensorflow as tf

w = tf.Variable(tf.ones(shape=(2,2)), name='w')
b = tf.Variable(tf.zeros(shape=(2)), name='b')

In [4]:
w

<tf.Variable 'w:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [5]:
b

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [7]:
@tf.function
def model(x):
    return w * x + b

out_a = model([1,0])
out_a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.]], dtype=float32)>

### 텐서플로 2.0 첫번째 예제

In [13]:
import tensorflow as tf
from tensorflow import keras as k

NB_CLASSES = 10
RESHAPED = 784
model = tf.keras.models.Sequential()
model.add(k.layers.Dense(NB_CLASSES, input_shape=(RESHAPED,),
                            kernel_initializer='zeros', name='dense_layer',
                            activation='softmax'))

In [14]:
model

### 예제 필기체 숫자 인식

In [203]:
import tensorflow as tf
import numpy as np
from tensorflow import keras as k

## 신경망과 훈련 매개변수 ## 
# epochs = 200
# batch_size = 128
# verbose = 1
# nb_classes = 10  # 출력 개수 = 숫자의 개수
# n_hidden = 128
# validation_split = .2  # 검증을 위해 남겨준 훈련 데이터

## mnist 데이터셋 로드 ##
mnist = k.datasets.mnist
(X_train,Y_train), (X_test,Y_test) = mnist.load_data()
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [204]:
## 60000 * 28 * 28 의 값을 60000 * 784 형태로 변환 ##
# reshaped = 784

X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')
X_train.shape, X_test.shape, X_train.dtype, X_test.dtype

((60000, 784), (10000, 784), dtype('float32'), dtype('float32'))

In [205]:
## 입력을 [0,1] 사이로 정규화 ##
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [206]:
## 모델 구축 ##
model = tf.keras.models.Sequential()
model.add(k.layers.Dense(10, input_shape=(784,), name='dense_layer', activation='softmax'))  # 활성화함수 : softmax

## 모델 요약 ##
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [207]:
## 모델 컴파일 ##
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# optimizer : 최적화기(SGD:확률적 그래디언트 하강) / loss : 목적함수 / metics : 척도(정확도,정밀도,재현율 등)

## 모델 훈련 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=200, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 1s 13us/sample - loss: 1.3795 - accuracy: 0.6730 - val_loss: 0.8915 - val_accuracy: 0.8311
Epoch 2/200
48000/48000 [==============================] - 0s 10us/sample - loss: 0.7895 - accuracy: 0.8307 - val_loss: 0.6543 - val_accuracy: 0.8602
Epoch 3/200
48000/48000 [==============================] - 0s 10us/sample - loss: 0.6400 - accuracy: 0.8535 - val_loss: 0.5589 - val_accuracy: 0.8718
Epoch 4/200
48000/48000 [==============================] - 0s 10us/sample - loss: 0.5680 - accuracy: 0.8637 - val_loss: 0.5063 - val_accuracy: 0.8780
Epoch 5/200
48000/48000 [==============================] - 0s 10us/sample - loss: 0.5242 - accuracy: 0.8701 - val_loss: 0.4726 - val_accuracy: 0.8846
Epoch 6/200
48000/48000 [==============================] - 0s 10us/sample - loss: 0.4943 - accuracy: 0.8750 - val_loss: 0.4485 - val_accuracy: 0.8888
Epoch 7/200
48000/48000 [=========================

In [208]:
## 모델 평가 ##
test_locc, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 16us/sample - loss: 0.2774 - accuracy: 0.9223
Test accuracy: 0.9223


In [209]:
model.save('model/mnist_1.h5')

#### 은닉층 추가로 모델 개선

In [193]:
import tensorflow as tf
import numpy as np
from tensorflow import keras as k

## 신경망과 훈련 매개변수 ## 
# epochs = 50
# batch_size = 128
# verbose = 1
# nb_classes = 10  # 출력 개수 = 숫자의 개수
# n_hidden = 128
# validation_split = .2  # 검증을 위해 남겨준 훈련 데이터

## mnist 데이터셋 로드 ##
mnist = k.datasets.mnist
(X_train,Y_train), (X_test,Y_test) = mnist.load_data()
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [194]:
## 60000 * 28 * 28 의 값을 60000 * 784 형태로 변환 후 [0,1] 사이로 정규화 ##
# reshaped = 784

X_train = X_train.reshape(60000, 784).astype('float32') / 255
X_test = X_test.reshape(10000, 784).astype('float32') / 255
X_train.shape, X_test.shape, X_train.dtype, X_test.dtype

((60000, 784), (10000, 784), dtype('float32'), dtype('float32'))

In [195]:
## 레이블을 원 핫 코드로 표기 ##
Y_train = tf.keras.utils.to_categorical(Y_train, 10)
Y_test = tf.keras.utils.to_categorical(Y_test, 10)
Y_train.shape, Y_test.shape, Y_train.dtype, Y_test.dtype

((60000, 10), (10000, 10), dtype('float32'), dtype('float32'))

In [196]:
## 모델 구축 ##
model = tf.keras.models.Sequential()
model.add(k.layers.Dense(128, input_shape=(784,), name='dence_layer', activation='relu'))
model.add(k.layers.Dense(128, name='dence_layer_2', activation='relu'))
model.add(k.layers.Dense(10, name='dence_layer_3', activation='softmax'))

## 모델 요약 ##
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dence_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dence_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dence_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [197]:
## 모델 컴파일 ##
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
# optimizer : 최적화기(SGD:확률적 그래디언트 하강) / loss : 목적함수 / metics : 척도(정확도,정밀도,재현율 등)

## 모델 훈련 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=50, verbose=1, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 1s 17us/sample - loss: 1.4308 - accuracy: 0.6385 - val_loss: 0.7198 - val_accuracy: 0.8449
Epoch 2/50
48000/48000 [==============================] - 1s 13us/sample - loss: 0.5836 - accuracy: 0.8533 - val_loss: 0.4448 - val_accuracy: 0.8863
Epoch 3/50
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4320 - accuracy: 0.8833 - val_loss: 0.3672 - val_accuracy: 0.9028
Epoch 4/50
48000/48000 [==============================] - 1s 13us/sample - loss: 0.3740 - accuracy: 0.8959 - val_loss: 0.3314 - val_accuracy: 0.9078
Epoch 5/50
48000/48000 [==============================] - 1s 13us/sample - loss: 0.3419 - accuracy: 0.9035 - val_loss: 0.3091 - val_accuracy: 0.9130
Epoch 6/50
48000/48000 [==============================] - 1s 14us/sample - loss: 0.3203 - accuracy: 0.9083 - val_loss: 0.2913 - val_accuracy: 0.9185
Epoch 7/50
48000/48000 [==============================] 

In [199]:
test_locc, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 22us/sample - loss: 0.1227 - accuracy: 0.9635
Test accuracy: 0.9635


In [202]:
model.save('model/mnist_2.h5')

#### 드롭아웃으로 단순망 개선

In [235]:
import tensorflow as tf
import numpy as np
from tensorflow import keras as k

#for tensorboard
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard('.logdir')

## 신경망과 훈련 매개변수 ## 
# epochs = 200
# batch_size = 128
# verbose = 1
# nb_classes = 10  # 출력 개수 = 숫자의 개수
# n_hidden = 128
# validation_split = .2  # 검증을 위해 남겨준 훈련 데이터
# dropout = .3

## mnist 데이터셋 로드 ##
mnist = k.datasets.mnist
(X_train,Y_train), (X_test,Y_test) = mnist.load_data()
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [236]:
## 60000 * 28 * 28 의 값을 60000 * 784 형태로 변환 ##
# reshaped = 784

X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')
X_train.shape, X_test.shape, X_train.dtype, X_test.dtype

((60000, 784), (10000, 784), dtype('float32'), dtype('float32'))

In [237]:
## 입력을 [0,1] 사이로 정규화 ##
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [238]:
## 레이블을 원 핫 코드로 표기 ##
Y_train = tf.keras.utils.to_categorical(Y_train, 10)
Y_test = tf.keras.utils.to_categorical(Y_test, 10)
Y_train.shape, Y_test.shape, Y_train.dtype, Y_test.dtype

((60000, 10), (10000, 10), dtype('float32'), dtype('float32'))

In [239]:
## 모델 구축 ##
model = tf.keras.Sequential()
model.add(k.layers.Dense(128, input_shape=(784,), name='dense_layer', activation='relu'))  # 활성화함수 : relu
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(128, name='dense_layer_2', activation='relu'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(10, name='dense_layer_3', activation='softmax'))

## 모델 요약 ##
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [240]:
## 모델 컴파일 ##
model.compile(optimizer='SGD',  # 최적화기
             loss='categorical_crossentropy',  # 목적함수
             metrics=['accuracy'])  # 척도

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=200, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 1s 23us/sample - loss: 1.6559 - accuracy: 0.4691 - val_loss: 0.8681 - val_accuracy: 0.8285
Epoch 2/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.8913 - accuracy: 0.7237 - val_loss: 0.5187 - val_accuracy: 0.8727
Epoch 3/200
48000/48000 [==============================] - 1s 16us/sample - loss: 0.6795 - accuracy: 0.7898 - val_loss: 0.4156 - val_accuracy: 0.8917
Epoch 4/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.5832 - accuracy: 0.8230 - val_loss: 0.3663 - val_accuracy: 0.9013
Epoch 5/200
48000/48000 [==============================] - 1s 16us/sample - loss: 0.5204 - accuracy: 0.8445 - val_loss: 0.3327 - val_accuracy: 0.9071
Epoch 6/200
48000/48000 [==============================] - 1s 15us/sample - loss: 0.4816 - accuracy: 0.8550 - val_loss: 0.3078 - val_accuracy: 0.9122
Epoch 7/200
48000/48000 [=========================

In [241]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 23us/sample - loss: 0.0704 - accuracy: 0.9785
Test accuracy: 0.9785


In [202]:
model.save('model/mnist_3(Adam).h5')

#### 컴파일 수정후 학습(목적함수 RMSProp)

In [218]:
## 모델 컴파일 ##
model.compile(optimizer='RMSProp',  # 최적화기
             loss='categorical_crossentropy',  # 목적함수
             metrics=['accuracy'])  # 척도

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=10, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1s 24us/sample - loss: 0.1078 - accuracy: 0.9666 - val_loss: 0.1012 - val_accuracy: 0.9726
Epoch 2/10
48000/48000 [==============================] - 1s 17us/sample - loss: 0.1005 - accuracy: 0.9681 - val_loss: 0.0961 - val_accuracy: 0.9739
Epoch 3/10
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0941 - accuracy: 0.9709 - val_loss: 0.0957 - val_accuracy: 0.9744
Epoch 4/10
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0878 - accuracy: 0.9732 - val_loss: 0.0988 - val_accuracy: 0.9755
Epoch 5/10
48000/48000 [==============================] - 1s 19us/sample - loss: 0.0840 - accuracy: 0.9738 - val_loss: 0.0928 - val_accuracy: 0.9766
Epoch 6/10
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0834 - accuracy: 0.9745 - val_loss: 0.0916 - val_accuracy: 0.9772
Epoch 7/10
48000/48000 [==============================] 

In [219]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 23us/sample - loss: 0.0843 - accuracy: 0.9792
Test accuracy: 0.9792


In [220]:
model.save('model/mnist_3(RMSProp).h5')

#### epoch 수 증가

In [50]:
## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=250, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/250
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0741 - accuracy: 0.9772 - val_loss: 0.0995 - val_accuracy: 0.9775
Epoch 2/250
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0671 - accuracy: 0.9792 - val_loss: 0.1024 - val_accuracy: 0.9776
Epoch 3/250
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0696 - accuracy: 0.9790 - val_loss: 0.1012 - val_accuracy: 0.9778
Epoch 4/250
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0676 - accuracy: 0.9806 - val_loss: 0.0980 - val_accuracy: 0.9793
Epoch 5/250
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0655 - accuracy: 0.9803 - val_loss: 0.1112 - val_accuracy: 0.9781
Epoch 6/250
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0632 - accuracy: 0.9807 - val_loss: 0.1097 - val_accuracy: 0.9783
Epoch 7/250
48000/48000 [=========================

In [51]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 22us/sample - loss: 0.3380 - accuracy: 0.9779
Test accuracy: 0.9779


#### 컴파일 교체(목적함수 Adam)

In [223]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=50, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 1s 26us/sample - loss: 0.0412 - accuracy: 0.9865 - val_loss: 0.0812 - val_accuracy: 0.9798
Epoch 2/50
48000/48000 [==============================] - 1s 16us/sample - loss: 0.0419 - accuracy: 0.9864 - val_loss: 0.0816 - val_accuracy: 0.9787
Epoch 3/50
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0402 - accuracy: 0.9868 - val_loss: 0.0778 - val_accuracy: 0.9805
Epoch 4/50
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0385 - accuracy: 0.9875 - val_loss: 0.0833 - val_accuracy: 0.9786
Epoch 5/50
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0360 - accuracy: 0.9882 - val_loss: 0.0907 - val_accuracy: 0.9780
Epoch 6/50
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0376 - accuracy: 0.9876 - val_loss: 0.0929 - val_accuracy: 0.9772
Epoch 7/50
48000/48000 [==============================] 

In [224]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 23us/sample - loss: 0.0936 - accuracy: 0.9805
Test accuracy: 0.9805


In [225]:
model.save('model/mnist_3(Adam).h5')

- 최적화기
- 목적함수 : Adam
- 드롭아웃 : 0.3

In [118]:
## 전개 ##
predictions = model.predict(X_test)
predictions.shape

(10000, 10)

In [119]:
predictions[0] # 7

array([1.2925690e-19, 1.8679447e-17, 8.8403975e-13, 4.5772324e-09,
       1.1199130e-22, 7.8696908e-15, 9.6924158e-29, 1.0000000e+00,
       3.2315000e-16, 2.5692367e-12], dtype=float32)

In [121]:
Y_test[0] # 7

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [ ]:
# 손글씨 데이터셋의 최적화기는 Adam
v1-0.9224, v2-0.9646, drop(0.3)-0.9779, RMSProp-(0.9768, 0.9779), Adam-0.9809

#### 내부 뉴런 증가(n_hidden)

In [126]:
## 모델 구축 ##
model = tf.keras.Sequential()
model.add(k.layers.Dense(128, input_shape=(784,), name='dense_layer', activation='relu'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(128, name='dense_layer_2', activation='relu'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(128, name='dense_layer_3', activation='relu'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(128, name='dense_layer_4', activation='relu'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(10, name='dense_layer_5', activation='softmax'))

## 모델 컴파일 ##
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=50, verbose=1, validation_split=.2)

In [127]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 24us/sample - loss: 0.0963 - accuracy: 0.9788
Test accuracy: 0.9788


- 증가량 미미함

#### batch_size 조절(128 -> 64)

In [129]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=64, epochs=50, verbose=1, validation_split=.2)

In [130]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 24us/sample - loss: 0.1157 - accuracy: 0.9785
Test accuracy: 0.9785


- 증가량 미미함

#### 활성화 함수 교체(sigmoid)

In [229]:
## 모델 구축 ##
model = tf.keras.Sequential()
model.add(k.layers.Dense(128, input_shape=(784,), name='dense_layer', activation='sigmoid'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(128, name='dense_layer_2', activation='sigmoid'))
model.add(k.layers.Dropout(.3))
model.add(k.layers.Dense(10, name='dense_layer_3', activation='softmax'))

## 모델 요약 ##
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_28 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_29 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [230]:
## 모델 컴파일 ##
model.compile(optimizer='SGD',  # 최적화기
             loss='categorical_crossentropy',  # 목적함수
             metrics=['accuracy'])  # 척도

## 모델 학습 ##
model.fit(X_train, Y_train,
         batch_size=128, epochs=200, verbose=1, validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 1s 21us/sample - loss: 2.3837 - accuracy: 0.1069 - val_loss: 2.2784 - val_accuracy: 0.1772
Epoch 2/200
48000/48000 [==============================] - 1s 16us/sample - loss: 2.3419 - accuracy: 0.1194 - val_loss: 2.2484 - val_accuracy: 0.2607
Epoch 3/200
48000/48000 [==============================] - 1s 16us/sample - loss: 2.3013 - accuracy: 0.1382 - val_loss: 2.2171 - val_accuracy: 0.4223
Epoch 4/200
48000/48000 [==============================] - 1s 16us/sample - loss: 2.2650 - accuracy: 0.1602 - val_loss: 2.1817 - val_accuracy: 0.4963
Epoch 5/200
48000/48000 [==============================] - 1s 14us/sample - loss: 2.2250 - accuracy: 0.1859 - val_loss: 2.1373 - val_accuracy: 0.5278
Epoch 6/200
48000/48000 [==============================] - 1s 15us/sample - loss: 2.1759 - accuracy: 0.2229 - val_loss: 2.0803 - val_accuracy: 0.5428
Epoch 7/200
48000/48000 [=========================

In [231]:
## 모델 평가 ##
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 22us/sample - loss: 0.2520 - accuracy: 0.9259
Test accuracy: 0.9259


#### 활성화함수 sigmoid 모델링 분류

In [ ]:
sigmoid, Adam, categorical_crossentropy, batch_size=64, 0.9805
sigmoid, Adam, categorical_crossentropy, batch_size=128, 0.98

sigmoid, RMSProp, categorical_crossentropy, batch_size=64, 0.9801
sigmoid, RMSProp, categorical_crossentropy, batch_size=128, 0.9809

sigmoid, SGD, categorical_crossentropy, batch_size=64, 0.9807
sigmoid, SGD, categorical_crossentropy, batch_size=128, 0.9814 # 최적화기

sigmoid, Adam, MSE, batch_size=64, 0.9802
sigmoid, Adam, MSE, batch_size=128, 0.9796

#### 모델 불러오기

In [245]:
model = k.models.load_model('model/mnist_3(Adam).h5')
model

#### 정규화 예제

In [251]:
tf.keras.regularizers.l1(l=0.01) # or 0.1 or 0.001
tf.keras.regularizers.l2(l=0.01) # or 0.1 or 0.001
tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01) # or 0.1 or 0.001

In [252]:
model = tf.keras.layers.Dense(
    128, input_dim=784,
    kernel_initializer='ones',
    kernel_regularizer=tf.keras.regularizers.l1(0.1),
    activity_regularizer=tf.keras.regularizers.l2(0.1))
model

#### learning rate 조정

In [255]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import SGD

# network and training
EPOCHS = 20
BATCH_SIZE = 256
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3
OPTMIZER = SGD(lr=0.1)

# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784
#
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

#one-hot
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

#build the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
   		input_shape=(RESHAPED,),
   		name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
   		name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
   		name='dense_layer_3', activation='softmax'))

# summary of the model
model.summary()

# compiling the model
model.compile(optimizer=OPTMIZER, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#training the moodel
model.fit(X_train, Y_train,
		batch_size=BATCH_SIZE, epochs=EPOCHS,
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

60000 train samples
10000 test samples
Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_34 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_35 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [

## 감정분석 예제

In [259]:
#!pip install tensorflow_datasets

In [290]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing
import tensorflow_datasets as tfds

# max_len = 200
# n_words = 10000
# dim_enbedding = 256
# epochs = 20
# batch_size = 500 < 보폭 or 학습률

def load_data():
    # 데이터 로드
    (X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=10000)
    # 문장을 max_len(200)이 되도록 채워 넣는다.
    X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=200)
    X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=200)
    return (X_train, y_train), (X_test, y_test)

In [291]:
def build_model():
    model = models.Sequential()
    # 입력 : eEmbedding layer
    # 모델은 크기의 정수 행렬을 입력으로 취한다(batch, input_length).
    # 모델의 출력은 차원이다(input_length, dim_embedding).
    # 입력 중 가장 큰 정수는 n_words(10000)보다 작거나 같다(어휘 크기).
    model.add(layers.Embedding(10000, 256, input_length=200))  # n_words = 10000, dim_embedding = 256, max_len = 200
    model.add(layers.Dropout(.3))
    
    # 각 n_words 특징에서 특징 벡터의 최댓값을 취한다.
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [292]:
(X_train, y_train), (X_test, y_test) = load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 200), (25000,), (25000, 200), (25000,))

In [265]:
X_train[0]

array([   5,   25,  100,   43,  838,  112,   50,  670,    2,    9,   35,
        480,  284,    5,  150,    4,  172,  112,  167,    2,  336,  385,
         39,    4,  172, 4536, 1111,   17,  546,   38,   13,  447,    4,
        192,   50,   16,    6,  147, 2025,   19,   14,   22,    4, 1920,
       4613,  469,    4,   22,   71,   87,   12,   16,   43,  530,   38,
         76,   15,   13, 1247,    4,   22,   17,  515,   17,   12,   16,
        626,   18,    2,    5,   62,  386,   12,    8,  316,    8,  106,
          5,    4, 2223, 5244,   16,  480,   66, 3785,   33,    4,  130,
         12,   16,   38,  619,    5,   25,  124,   51,   36,  135,   48,
         25, 1415,   33,    6,   22,   12,  215,   28,   77,   52,    5,
         14,  407,   16,   82,    2,    8,    4,  107,  117, 5952,   15,
        256,    4,    2,    7, 3766,    5,  723,   36,   71,   43,  530,
        476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,  104,
         88,    4,  381,   15,  297,   98,   32, 20

In [272]:
## 설정한 함수를 이용해 모델 정의하기 ##
model = build_model()
model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 256)          2560000   
_________________________________________________________________
dropout_36 (Dropout)         (None, 200, 256)          0         
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_37 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
___________________________________________

In [276]:
## 모델 컴파일링 ##
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [277]:
## 모델 학습 ##
score = model.fit(X_train, y_train,
                 epochs=20, batch_size=500, validation_data=(X_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 14s 576us/sample - loss: 0.6715 - accuracy: 0.6174 - val_loss: 0.6271 - val_accuracy: 0.8381
Epoch 2/20
25000/25000 [==============================] - 15s 610us/sample - loss: 0.4564 - accuracy: 0.8405 - val_loss: 0.3620 - val_accuracy: 0.8591
Epoch 3/20
25000/25000 [==============================] - 14s 549us/sample - loss: 0.2792 - accuracy: 0.8856 - val_loss: 0.3062 - val_accuracy: 0.8754
Epoch 4/20
25000/25000 [==============================] - 14s 546us/sample - loss: 0.2150 - accuracy: 0.9172 - val_loss: 0.2899 - val_accuracy: 0.8790
Epoch 5/20
25000/25000 [==============================] - 14s 552us/sample - loss: 0.1705 - accuracy: 0.9374 - val_loss: 0.2879 - val_accuracy: 0.8779
Epoch 6/20
25000/25000 [==============================] - 14s 554us/sample - loss: 0.1336 - accuracy: 0.9554 - val_loss: 0.2949 - val_accuracy: 0.8728
Epoch 7/20
25000/25000 [====================

TypeError: 'History' object is not subscriptable

In [286]:
score = model.evaluate(X_test, y_test, batch_size=500)
print('\nTest score:', score[0])
print('Test accuracy:', score[1])

25000/25000 [==============================] - 1s 49us/sample - loss: 0.5019 - accuracy: 0.8515

Test score: 0.5018726927042008
Test accuracy: 0.85148


In [287]:
model.save('model/imdb.h5')

In [288]:
## 예측하기 ##
'''prediction_1 = model.predict(X) # X : 예측할 데이터
prediction_2 = model.predict_proba(X) # X : 예측할 데이터'''

'prediction_1 = model.predict(X) # X : 예측할 데이터\nprediction_2 = model.predict_proba(X) # X : 예측할 데이터'